In [1]:
import tifffile
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np
import os

sc_root_dir = "/import/home/share/zw/data/breast_cancer"
st_root_dir = "/import/home/share/zw/data/breast_cancer/paper_data/downstream_xenium_breast_cancer/scVI_output"
save_path = "/import/home/share/zw/pql/data/breast_cancer"

Since the new h5ad version, can only be run with pql environment.

In [2]:
# # scRNA
# adata_sc = sc.read_h5ad(f"{sc_root_dir}filtered_sc.h5ad")
# adata_sc

In [3]:
# PCA + UMAP
# sc.pp.pca(adata_sc)
# sc.pp.neighbors(adata_sc)
# sc.tl.umap(adata_sc)

# sc.pl.umap(adata_sc, color='celltype')

In [4]:
# counts_sc = pd.DataFrame(adata_sc.X.toarray().astype(int),index=adata_sc.obs.index,columns=adata_sc.var_names)
# counts_sc.to_csv(os.path.join(save_path, "rawdata", "sc_counts.csv"))

In [5]:
# adata_sc.obs.to_csv(os.path.join(save_path, "rawdata", "sc_celltype.csv"))

In [9]:
# ST  adata with scanvi label and spatial coordinates of mask center
# method = "Cell_10X"
method = "UCS_10X"
adata_st = sc.read_h5ad(os.path.join(st_root_dir, method, "annotated_adata_st.h5ad"))

In [10]:
if not os.path.exists(os.path.join(save_path, method)):
    os.makedirs(os.path.join(save_path, method))
if not os.path.exists(os.path.join(save_path, "rawdata", method)):
    os.makedirs(os.path.join(save_path, "rawdata", method))

In [11]:
adata_st

AnnData object with n_obs × n_vars = 165734 × 307
    obs: 'tech', '_scvi_batch', '_scvi_labels', 'celltype_scanvi', 'C_scANVI'
    uns: 'C_scANVI_colors', '_scvi_manager_uuid', '_scvi_uuid', 'log1p', 'tech_colors'
    obsm: 'X_mde_scanvi', 'X_scANVI', 'X_scVI', 'X_scVI_mde', 'spatial'
    layers: 'counts'

In [24]:
adata_st.obs

,tech,_scvi_batch,_scvi_labels,celltype_scanvi,C_scANVI
1,st,1,17,Unknown,DCIS 2
2,st,1,17,Unknown,DCIS 2
3,st,1,17,Unknown,DCIS 2
4,st,1,17,Unknown,Invasive Tumor
5,st,1,17,Unknown,DCIS 2
...,...,...,...,...,...
167776,st,1,17,Unknown,DCIS 1
167777,st,1,17,Unknown,DCIS 1
167778,st,1,17,Unknown,DCIS 1
167779,st,1,17,Unknown,DCIS 1


In [25]:
# adata_st.X[1,0:20].toarray()
# adata_st.layers["counts"][1,0:20].toarray()
# adata_st.obsm['spatial']

In [26]:
n_spots = adata_st.n_obs
n_samples = 10000
random_indices = np.random.choice(n_spots, size=n_samples, replace=False)
adata_st_subset = adata_st[random_indices]

In [ ]:
counts = pd.DataFrame(adata_st_subset.layers["counts"].toarray().astype(int),index=adata_st_subset.obs.index,columns=adata_st_subset.var_names)
counts.to_csv(os.path.join(save_path, "rawdata", method, "counts.csv"))

In [28]:
adata_st_subset.obsm['spatial']

ArrayView([[1682.,  106.],
           [4447.,  153.],
           [1454.,  364.],
           ...,
           [  35., 1855.],
           [ 783.,  173.],
           [2983., 5174.]])

In [29]:
coordinates = pd.DataFrame(adata_st_subset.obsm['spatial'].toarray(),index=adata_st_subset.obs.index, columns=["x","y"])
coordinates.to_csv(os.path.join(save_path, "rawdata", method, "coordinates.csv"))

In [30]:
scvi_label = pd.DataFrame(adata_st_subset.obs["C_scANVI"],index=adata_st_subset.obs.index)
scvi_label.columns = ["celltype"]
scvi_label.to_csv(os.path.join(save_path, "rawdata", method, "scvi_label.csv"))